# Napari image viewer

Uses the octopuslite dask image loader to automatically sort and load images from multichannel time lapse experiments. Can also use btrack to load associated tracks and segmentation images.

In [ ]:
import napari
from octopuslite import DaskOctopusLiteLoader
import glob
import numpy as np
from skimage.io import imread
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
expt_info = pd.read_csv('/home/nathan/data/kraken/ras/experiment_info_april22.csv', header = [1])

In [ ]:
expt_info

In [ ]:
expt_info.loc[(expt_info['Useable (in radial analysis)'] == True)]

In [ ]:
expt_pos_list = expt_info.loc[(expt_info['Useable (in radial analysis)'] ==  True)][['EXP n˚','POSITION', 'CELL TYPE']]


In [ ]:
expt_pos_list

In [ ]:
root_dir = '/home/nathan/data/kraken/ras'
viewer = napari.Viewer()
for i, expt_pos in tqdm(expt_pos_list.iterrows(), desc = 'Progress throughout valid expts', total = len(expt_pos_list)):
    expt = expt_pos['EXP n˚']
    pos = expt_pos['POSITION']
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    images = DaskOctopusLiteLoader(image_path)
#     stack_gfp = []
#     for i in tqdm(images.files('gfp'), total = len(images.files('gfp'))):
#         img_gfp = imread(i)
#         stack_gfp.append(img_gfp)
#     stack_gfp = np.stack(stack_gfp, axis = 0)
    stack_bf = []
    for i in tqdm(images.files('brightfield'), total = len(images.files('gfp'))):
        img_bf = imread(i)
        stack_bf.append(img_bf)
    stack_bf = np.stack(stack_bf, axis = 0)
    stack_rfp = []
    for i in tqdm(images.files('rfp'), total = len(images.files('rfp'))):
        img_rfp = imread(i.replace('channel001', 'channel002'))
        stack_rfp.append(img_rfp)
    stack_rfp = np.stack(stack_rfp, axis = 0)
#     viewer.add_image(stack_gfp, name=f"gfp {expt}, {pos}", 
#                      contrast_limits=[0,255],
#                      blending = 'additive', colormap = 'green')
    viewer.add_image(stack_bf, name=f"bf {expt}, {pos}", 
                     contrast_limits=[0,255],
                     blending = 'additive')
    viewer.add_image(stack_rfp, name=f"rfp {expt}, {pos}", 
                     contrast_limits=[0,55], 
                     blending = 'additive', colormap = 'magenta')
    viewer.dims.current_step = (0,0,0)
    viewer.show(block=True)
    
    break

In [ ]:
stack_rfp = []
for i in tqdm(images.files('rfp'), total = len(images.files('rfp'))):
    img_rfp = imread(i.replace('channel001', 'channel002'))
    stack_rfp.append(img_rfp)
stack_rfp = np.stack(stack_rfp, axis = 0)

In [ ]:
viewer.add_image(stack_rfp, name=f"rfp {expt}, {pos}", 
                 contrast_limits=[0,55], 
                 blending = 'additive', colormap = 'magenta')

# Load images

In [ ]:
# define experiment ID and select a position
expt = 'ND0013'
pos = 'Pos7'
# point to where the data is

image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'

In [ ]:
# lazily load images
images = DaskOctopusLiteLoader(image_path)

In [ ]:
files = images.files('gfp')

In [ ]:
stack_gfp = []
for i in tqdm(images.files('gfp'), total = len(images.files('gfp'))):
    img_gfp = imread(i)
    stack_gfp.append(img_gfp)
stack_gfp = np.stack(stack_gfp, axis = 0)
stack_rfp = []
for i in tqdm(images.files('rfp'), total = len(images.files('rfp'))):
    img_rfp = imread(i.replace('channel001', 'channel002'))
    stack_rfp.append(img_rfp)
stack_rfp = np.stack(stack_rfp, axis = 0)

In [ ]:
viewer = napari.Viewer()

viewer.add_image(stack_gfp, name="gfp", 
                 contrast_limits=[0,255],
                 blending = 'additive', colormap = 'green')
viewer.add_image(stack_rfp, name="rfp", 
                 contrast_limits=[0,55], 
                 blending = 'additive', colormap = 'magenta')

# Edit list of coords post finding

In [ ]:
transform_tensor = np.load(transform_path)
img = imread(files[0])
shape = img.shape
dims = img.ndim
cslice = lambda d: slice(
    int((shape[d] - crop[d]) // 2), int((shape[d] - crop[d]) // 2 + crop[d])
)
crops = tuple([cslice(d) for d in range(dims)])